In [64]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize



In [52]:
train_data = pd.read_csv('train.csv')

In [54]:
count_vectorizer = CountVectorizer()
feature_vector = count_vectorizer.fit(train_data['reviews_content'])
features = count_vectorizer.get_feature_names_out()
print(len(count_vectorizer.get_feature_names_out()))
features = list(features)
print(random.sample(features,100))

35177
['volcanoes', 'glistening', 'parable', 'burp', 'interminably', 'moriarty', 'badlands', 'prevent', 'canine', 'flooding', 'juliette', 'disjointment', 'dire', 'sighing', 'leisure', 'hygienists', 'alum', 'hortense', 'negotiators', 'petstore', 'sleight', 'choked', 'raise', '1951', 'thrax', 'cynically', 'slobby', 'oranges', 'admire', 'modernizing', 'pacification', 'snoozing', 'happenings', 'braveheart', 'jazz', 'nipples', 'watts', 'lump', 'engaged', 'thunk', 'tumbling', 'spunky', 'climate', 'rapper', 'faceted', 'shaker', 'demonic', 'dexterity', 'irrefutable', 'lifeforms', 'int', 'perfect', 'katanga', 'crunchily', 'derailing', 'howled', 'varley', 'behold', 'cure', 'turns', 'suspenseless', 'uncontrollably', 'band', 'gunk', 'gumpian', 'vengeful', 'crud', 'directorial', 'bungalow', 'taunt', 'earning', 'doves', 'siouxsie', 'carousing', 'coping', 'caller', 'healed', 'engineers', 'slew', 'ac3', 'teaches', 'delegation', 'desk', 'phases', 'prevents', 'hell', 'mistaken', 'blowjobs', 'categorize'

In [53]:
train_data_MatVec = count_vectorizer.transform(train_data['reviews_content'])
train_data_MatVec.getnnz()
print("Density of the matrix: ", train_data_MatVec.getnnz()*100/
               (train_data_MatVec.shape[0]*train_data_MatVec.shape[1]))

Density of the matrix:  0.936727596630374


In [55]:
train_data_Vec_df = pd.DataFrame(train_data_MatVec.todense())
train_data_Vec_df.columns = features
train_data_Vec_df[['important','fix','it']][0:20]

,important,fix,it
0,1,0,8
1,0,0,12
2,1,0,2
3,0,0,12
4,0,0,7
5,0,0,9
6,0,0,9
7,0,0,4
8,0,0,3
9,0,0,2


In [38]:
features_counts = np.sum(train_data_MatVec.toarray(),axis=0)
features_counts_df = pd.DataFrame(dict(features = features, counts = features_counts))
features_counts_df = features_counts_df.sort_values('counts',ascending=False)

# features_counts_df.counts.sort_values(ascending= False)

# x = features_counts_df.sort_values('counts',ascending=False)
# for i in range(100):
#     print(x.iloc[i])

# plt.figure(figsize=(8,6))
# plt.hist(features_counts_df.counts)
# plt.xlabel('Frequency of Words')
# plt.ylabel('Density')
# plt.show()

In [39]:
features_counts_df.to_csv('features_counts_df_2.csv',index = False)


In [40]:
from sklearn.feature_extraction import text
my_stop_words = text.ENGLISH_STOP_WORDS
print("List of stop words: ", (my_stop_words))

List of stop words:  frozenset({'find', 'five', 'here', 'please', 'on', 'top', 'thick', 'latter', 'into', 'your', 'out', 'indeed', 'under', 'before', 'formerly', 'cant', 'etc', 'other', 'alone', 'fire', 'should', 'beside', 'one', 'beforehand', 'she', 'have', 'thru', 'six', 'bottom', 'was', 'such', 'too', 'for', 'less', 'her', 'as', 'latterly', 'itself', 'off', 'onto', 'beyond', 'get', 'amongst', 'anywhere', 'can', 'thereby', 'herein', 'mine', 'found', 'anything', 'therein', 'therefore', 'sometimes', 'eleven', 'is', 'wherein', 'hundred', 'he', 'made', 'hereby', 'co', 'they', 'also', 'from', 'among', 'upon', 'amount', 'us', 'empty', 'mostly', 'their', 'then', 'what', 'ever', 'were', 'whether', 'wherever', 'when', 'due', 'much', 'give', 'ltd', 'already', 'nor', 'every', 'seeming', 'call', 'however', 'full', 'behind', 'even', 'some', 're', 'toward', 'those', 'his', 'forty', 'over', 'namely', 'everything', 'describe', 'because', 'keep', 'becoming', 'now', 'would', 'con', 'without', 'wheneve

In [41]:
my_stop_words = set(my_stop_words)
excluded_words = {'not','nor','cannot','neither','more','whereas','mostly','very'}
my_stop_words = my_stop_words - excluded_words

# Reprocess

In [18]:
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')

In [19]:
df_train 

,reviews_content,category
0,airplane ! is considered among many to be the ...,positive
1,you've got to love disney . \nno matter what t...,positive
2,""" the tailor of panama "" is a different kind ...",positive
3,"the characters in jonathan lynn's "" the whole ...",negative
4,"vikings v . bears ? \nno , this isn't the line...",negative
...,...,...
1495,"trekkies , roger nygard's energetic and hilari...",positive
1496,""" dangerous beauty "" is a really nothing more...",positive
1497,starring shawnee smith ; donovan leitch ; rick...,negative
1498,"man , this was one wierd movie . \nsimilar to ...",negative


In [42]:
# Function to remove stop words
def remove_stop_words(text):
    # Tokenize the text
    words = text.split()
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in my_stop_words]
    # Join the remaining words back into a single string
    processed_text = ' '.join(filtered_words)
    return processed_text
def remove_line_break(text):
    words = text.split()
    output = [element.replace("\n","") for element in words]
    output = ' '.join(output)
    return output
def extract_alphabetic(text):
    # Use regular expression to remove non-alphabetic characters
    return re.sub(r'[^a-zA-Z\s]', '', text)

df_train['Processed Contents'] = df_test['reviews_content'].apply(lambda x: [element.replace("\n", "") for element in x.split(' ')])

In [43]:
df_train['Processed Contents'] = df_train['reviews_content'].apply(remove_line_break)

df_train['Processed Contents'] = df_train['Processed Contents'].apply(remove_stop_words)

In [44]:
df_train

,reviews_content,category,Processed Contents
0,airplane ! is considered among many to be the ...,positive,airplane ! considered epitome satire film-maki...
1,you've got to love disney . \nno matter what t...,positive,"you've got love disney . matter serve , guaran..."
2,""" the tailor of panama "" is a different kind ...",positive,""" tailor panama "" different kind spy movie . d..."
3,"the characters in jonathan lynn's "" the whole ...",negative,"characters jonathan lynn's "" yards , "" endless..."
4,"vikings v . bears ? \nno , this isn't the line...",negative,"vikings v . bears ? , isn't lineup monday nigh..."
...,...,...,...
1495,"trekkies , roger nygard's energetic and hilari...",positive,"trekkies , roger nygard's energetic hilarious ..."
1496,""" dangerous beauty "" is a really nothing more...",positive,""" dangerous beauty "" really more grandiose soa..."
1497,starring shawnee smith ; donovan leitch ; rick...,negative,starring shawnee smith ; donovan leitch ; rick...
1498,"man , this was one wierd movie . \nsimilar to ...",negative,"man , wierd movie . similar conspiracy theory ..."


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [109]:
TFIDF_Vectorizer = TfidfVectorizer()
TFIDF_Vectorizer.fit(df_train['reviews_content'])
X_train = TFIDF_Vectorizer.transform(df_train['reviews_content'])
y_train = df_train['category']
df_test = pd.read_csv('test.csv')
X_test = TFIDF_Vectorizer.transform(df_test['reviews_content'])
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, df_train['category'], test_size=0.1)


In [47]:
svm_model  = SVC(kernel = 'linear')
X.shape

(1500, 35177)

In [50]:
svm_model.fit(X_train,y_train)
y_pred_train = svm_model.predict(X_train)
y_pred_val = svm_model.predict(X_val)
print("train error :",accuracy_score(y_train,y_pred_train))
print("val error :",accuracy_score(y_val,y_pred_val))

train error : 0.9874074074074074
val error : 0.8266666666666667


In [65]:

df_train = pd.read_csv('train.csv')
#This code concatenates all the text from the reviews_content column in the DataFrame df into a single string, with each review separated by a space.
reviews = df_train.reviews_content.str.cat(sep=' ')
tokens = word_tokenize(reviews)
vocabulary = set(tokens)
print(len(vocabulary))

40463


In [67]:
frequency_dist = nltk.FreqDist(tokens)
frecu

list

In [96]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred_val = clf.predict(X_val)
print("train error :",accuracy_score(y_train,y_pred_train))
print("val error :",accuracy_score(y_val,y_pred_val))

train error : 0.968
val error : 0.98


In [97]:

clf = MultinomialNB()
clf.fit(X_train,y_train)
y_pred_test = clf.predict(X_test)
df_output = pd.DataFrame({'Row': range(1,501),'Label':y_pred_test})
print(dir(pd))
df_output.to_csv('MultinomialNaiveBayes.csv',index=False)

['ArrowDtype', 'BooleanDtype', 'Categorical', 'CategoricalDtype', 'CategoricalIndex', 'DataFrame', 'DateOffset', 'DatetimeIndex', 'DatetimeTZDtype', 'ExcelFile', 'ExcelWriter', 'Flags', 'Float32Dtype', 'Float64Dtype', 'Grouper', 'HDFStore', 'Index', 'IndexSlice', 'Int16Dtype', 'Int32Dtype', 'Int64Dtype', 'Int8Dtype', 'Interval', 'IntervalDtype', 'IntervalIndex', 'MultiIndex', 'NA', 'NaT', 'NamedAgg', 'Period', 'PeriodDtype', 'PeriodIndex', 'RangeIndex', 'Series', 'SparseDtype', 'StringDtype', 'Timedelta', 'TimedeltaIndex', 'Timestamp', 'UInt16Dtype', 'UInt32Dtype', 'UInt64Dtype', 'UInt8Dtype', '__all__', '__builtins__', '__cached__', '__doc__', '__docformat__', '__file__', '__git_version__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_built_with_meson', '_config', '_is_numpy_dev', '_libs', '_pandas_datetime_CAPI', '_pandas_parser_CAPI', '_testing', '_typing', '_version_meson', 'annotations', 'api', 'array', 'arrays', 'bdate_range', 'compat', 'conca

In [110]:
from sklearn.naive_bayes import GaussianNB
# Cross Valdation
model_gaussianNB = GaussianNB()
model_gaussianNB.fit(X_train.toarray(),y_train)
y_pred_val = model_gaussianNB.predict(X_val.toarray())
y_pred_train = model_gaussianNB.predict(X_train.toarray())
print("train error :",accuracy_score(y_train,y_pred_train))
print("val error :",accuracy_score(y_val,y_pred_val))
# real
model_gaussianNB = GaussianNB()
model_gaussianNB.fit(X_train.toarray(),y_train)
y_pred_test = model_gaussianNB.predict(X_test.toarray())
df_output = pd.DataFrame({'Row': range(1,501),'Label':y_pred_test})
df_output.to_csv('GaussianNaiveBayes.csv',index=False)

train error : 0.9992592592592593
val error : 0.6333333333333333
